In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import altair as alt


import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

plt.rcParams['svg.fonttype'] = 'none'

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


In [4]:
df=pd.read_csv("merged_data.csv")
df.head()

merged_data
Month 2023_temp 2022_temp 2021_temp 2020_temp 2023_humid 2022_humid 2021_humid 2020_humid date    class                    focuses uf          2020_rain 2021_rain 2022_rain   2023_rain
4.0   26.4      25.7      26.0      26.2      87.6       90.3       89.1       88.2       2023/04 Desmatamento Consolidado 8.0     AMAZONAS    974.8     1115.5    1240.2          349.6
                                                                                                                           9.0     MARANHÃO    974.8     1115.5    1240.2          349.6
                                                                                                                           271.0   MATO GROSSO 974.8     1115.5    1240.2          349.6
                                                                                                                           37.0    PARÁ        974.8     1115.5    1240.2          349.6

In [8]:
# Correctly loading the CSV file now that we've confirmed the structure and the delimiter
data = pd.read_csv('merged_data.csv', skiprows=1)  # Skipping the first row which is just 'merged_data'

# Confirming the correct data structure this time
data.head()

,Month,2023_temp,2022_temp,2021_temp,2020_temp,2023_humid,2022_humid,2021_humid,2020_humid,date,class,focuses,uf,2020_rain,2021_rain,2022_rain,2023_rain
0,4.0,26.4,25.7,26.0,26.2,87.6,90.3,89.1,88.2,2023/04,Desmatamento Consolidado,8.0,AMAZONAS,974.8,1115.5,1240.2,349.6
1,4.0,26.4,25.7,26.0,26.2,87.6,90.3,89.1,88.2,2023/04,Desmatamento Consolidado,9.0,MARANHÃO,974.8,1115.5,1240.2,349.6
2,4.0,26.4,25.7,26.0,26.2,87.6,90.3,89.1,88.2,2023/04,Desmatamento Consolidado,271.0,MATO GROSSO,974.8,1115.5,1240.2,349.6
3,4.0,26.4,25.7,26.0,26.2,87.6,90.3,89.1,88.2,2023/04,Desmatamento Consolidado,37.0,PARÁ,974.8,1115.5,1240.2,349.6
4,4.0,26.4,25.7,26.0,26.2,87.6,90.3,89.1,88.2,2023/04,Desmatamento Consolidado,15.0,RONDÔNIA,974.8,1115.5,1240.2,349.6


In [16]:
%%R 
data <- read_csv("merged_data.csv", skip = 1)

# Selecting predictors and the dependent variable
data_selected <- select(data, focuses, `2023_temp`, `2023_humid`, `2023_rain`)

# Checking for NA values and removing them if necessary
data_selected <- na.omit(data_selected)

# Linear regression with 'focuses' as the dependent variable
model <- lm(focuses ~ ., data = data_selected)

# Summary of the model to check coefficients and statistics
summary(model)


Rows: 2040 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): date, class, uf
dbl (14): Month, 2023_temp, 2022_temp, 2021_temp, 2020_temp, 2023_humid, 202...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Call:
lm(formula = focuses ~ ., data = data_selected)

Residuals:
   Min     1Q Median     3Q    Max 
-487.4 -293.2  -68.7   20.1 7376.1 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)  
(Intercept)   257.4388  2770.7691   0.093   0.9260  
`2023_temp`    58.3843    69.7494   0.837   0.4027  
`2023_humid`  -19.2564    11.3396  -1.698   0.0896 .
`2023_rain`    -0.1086     0.1318  -0.824   0.4102  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 608.6 on 2035 degrees of freedom
Multiple R-squared:  0.09328,	Adjusted R-squared:  0.09194 
F-statistic: 69

In [11]:
%%R
# Assuming 'data' is your dataframe
data_long <- data %>% 
  gather(key = "variable", value = "value", -focuses, -uf, -class, -date) %>%
  separate(variable, into = c("year", "measure"), sep = "_") %>%
  spread(key = measure, value = value)

# Convert year to a factor for the regression analysis
data_long$year <- as.factor(data_long$year)

# Now fit a linear model with focuses as the dependent variable
model <- lm(focuses ~ year + temp + humid + rain, data = data_long)
summary(model)



Call:
lm(formula = focuses ~ year + temp + humid + rain, data = data_long)

Residuals:
   Min     1Q Median     3Q    Max 
-541.7 -256.3 -105.8   36.5 7384.4 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) 2349.19813  860.71830   2.729  0.00636 ** 
year2021     102.01904   19.75267   5.165 2.46e-07 ***
year2022      -7.51587   19.74884  -0.381  0.70353    
year2023      20.40409   25.29195   0.807  0.41984    
temp          12.78619   20.90382   0.612  0.54077    
humid        -30.91410    3.80942  -8.115 5.56e-16 ***
rain           0.06842    0.01172   5.839 5.45e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 611.7 on 8149 degrees of freedom
  (2044 observations deleted due to missingness)
Multiple R-squared:  0.08301,	Adjusted R-squared:  0.08233 
F-statistic: 122.9 on 6 and 8149 DF,  p-value: < 2.2e-16



In addition: Warning message:
Expected 2 pieces. Missing pieces filled with `NA` in 2040 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...]. 


In [12]:
%%R 

# Assuming 'data_long' is already your reshaped dataset including the 'class' variable
model_with_class <- lm(focuses ~ class + temp + humid + rain, data = data_long)
summary(model_with_class)



Call:
lm(formula = focuses ~ class + temp + humid + rain, data = data_long)

Residuals:
   Min     1Q Median     3Q    Max 
-785.8 -257.8  -88.4   93.6 7200.4 

Coefficients:
                            Estimate Std. Error t value Pr(>|t|)    
(Intercept)                1.862e+03  7.096e+02   2.624   0.0087 ** 
classDesmatamento Recente  1.822e+02  2.019e+01   9.025  < 2e-16 ***
classFloresta             -1.308e+02  2.045e+01  -6.399 1.65e-10 ***
classOutros               -2.464e+02  2.064e+01 -11.937  < 2e-16 ***
classVegetacao Secundaria -2.096e+02  2.027e+01 -10.341  < 2e-16 ***
temp                       2.242e+01  1.668e+01   1.345   0.1787    
humid                     -2.711e+01  3.268e+00  -8.296  < 2e-16 ***
rain                       8.031e-02  8.848e-03   9.077  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 593 on 8148 degrees of freedom
  (2044 observations deleted due to missingness)
Multiple R-squared:  0.1382,	A